In [ ]:
import tensorflow as ts
from tensorflow import keras
import numpy as np
import tensorflow.keras.backend as K
from keras.models import Sequential
from keras.layers import Dense, InputLayer
from keras.callbacks import Callback

def custom_loss(y_true, y_pred):
    """
    Calculates the L2 norm of the difference between y_pred and a constant value, plus
    a scaled sum of y_pred, for each example in the batch. Returns the average L2 norm
    over all examples.

    Args:
    y_true: tensor of true labels
    y_pred: tensor of predicted labels

    Returns:
    A tensor representing the mean L2 norm over all examples.
    """
    
    a, beta, e, x0, T = 0.0, 2.0, 1.0, 5.0, 1.0
    
    # calculate the difference y_pred[i] - c * sum(y_pred)
    n = K.int_shape(y_pred)[1]
    hs = 1.0 / (n+1)
    diff = beta * e * (T + x0 - a) - y_pred - hs * beta * e * K.sum(y_pred, axis=1, keepdims=True)
    
    # calculate the L2 norm for each example
    norma = K.sqrt(K.sum(K.square(diff), axis=1))
    
    # return the average L2 norm over all training examples
    return K.mean(norma)



class SavePredictions(Callback):
    def __init__(self, x_val):
        self.x_val = x_val
        self.predictions = []
    
    def on_epoch_end(self, epoch, logs={}):
        # Faz as previsões no conjunto de validação
        y_pred = self.model.predict(self.x_val)
        
        # Salva as previsões em uma lista
        self.predictions.append(y_pred)
        
        # Salva as previsões em um arquivo a cada 5 épocas
        if epoch % 10 == 0:
            np.save('predictions_epoch{}.npy'.format(10000+epoch), y_pred)




In [ ]:
# Tamanho da entrada
n = 100

# Número de neurônios nas camadas ocultas
n1 = 5
# n2 = 2


# Criação do modelo
model = Sequential()

# Adicionando as camadas ao modelo
model.add(InputLayer(input_shape=(n,)))
model.add(Dense(n1, activation='relu')
# model.add(Dense(n2, activation='relu'))
model.add(Dense(n, activation='linear'))

# Compilando o modelo
model.compile(loss=custom_loss, optimizer='adam')

# Dados de treinamento
X_train = np.linspace(0,1,n).reshape(1,n)
y_train = np.linspace(0,1,n).reshape(1,n)

# Treinando o modelo
save_predictions = SavePredictions(X_train)
model.fit(X_train, y_train, epochs=1001, callbacks=[save_predictions])

# Fazendo uma previsão
y_pred = model.predict(X_train).ravel()

In [ ]:
import os
import seaborn as sns
import matplotlib.pyplot as plt
import imageio

# Define o número de iterações
directory = os.getcwd()
extensao = ".npy"
files = os.listdir()

filtered_files = sorted([file for file in files if file.endswith(extensao)])
num_iter = len(filtered_files)

# Loop pelas iterações
for i in range(num_iter):
    data = np.load(filtered_files[i])
    # Cria os dados do gráfico
    x = np.linspace(0, 1,n)
    y = data.ravel()
    
    # Cria a figura e o eixo
    fig, ax = plt.subplots()
    sns.lineplot(x=x, y=y, ax=ax)
    ax.set(xlim=[0,1],ylim=[-1,5])
    
    # Adiciona o número da iteração no canto superior direito
    ax.text(0.95, 0.95, f'Iteração: {i}', transform=ax.transAxes, ha='right', va='top')
    
    # Salva a figura em uma lista
    plt.savefig(f'iteracao_{i}.png', dpi=300)
    plt.close()

# Cria o gif a partir das imagens salvas
images = []
for i in range(num_iter):
    images.append(imageio.imread(f'iteracao_{i}.png'))
imageio.mimsave('animacao.gif', images, fps=5)
